In [1]:
import os
import sys
import numpy as np

# local imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import data_loader
#import plotting

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../experiments')))
import utils

2025-04-07 17:54:50.679357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 17:54:50.691895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744041290.706177  528677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744041290.710775  528677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 17:54:50.726917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
emb_dir = '../../../artifacts/embeddings/'

# huggingface model:
model = 'bert-base-uncased'
pooling = 'mean'
batch_size = 8

# openai model:
#model = 'text-embedding-3-small'
#pooling = ''

### Example 1: Big dataset with train-dev-test split

In [10]:
dataset_name = 'jigsaw'
local_dir = '../../../data/jigsaw_bias'

dataset = utils.get_dataset_with_embeddings(emb_dir, dataset_name, model, pooling, batch_size, local_dir)

X_train, emb_train, y_train, g_train, class_weights, group_weights = dataset.get_split('train')
X_dev, emb_dev, y_dev, g_dev, _, _ = dataset.get_split('dev')
X_test, emb_test, y_test, g_test, _, _ = dataset.get_split('test')

# X_* texts
# emb_* embeddings
# y_* class labels
# g_* protected group labels (one-hot encoded)

print("classes: ", dataset.class_names)
print("protected groups: ", dataset.group_names)

load JigsawBias with option: single-class


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
embed train split...
load precomputed embeddings for train set
embed dev split...
load precomputed embeddings for dev set
embed test split...
load precomputed embeddings for test set
classes:  ['not toxic', 'toxic']
protected groups:  ['female', 'male', 'transgender', 'white', 'asian', 'black', 'latino', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim', 'heterosexual', 'bisexual', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'physical_disability', 'psychiatric_or_mental_illness']


### Example 2: Small dataset with cross-validation

In [14]:
dataset_name = 'crows_pairs'
local_dir = None

dataset = utils.get_dataset_with_embeddings(emb_dir, dataset_name, model, pooling, batch_size, local_dir)
print("available splits: ", dataset.splits)

for fold_id in range(dataset.n_folds):
    data_dict = dataset.get_cv_split(fold_id)
    X_train, emb_train, y_train, g_train, cw, gw = data_dict['train']
    X_test, emb_test, y_test, g_test, _, _ = data_dict['test']

    # X_* texts
    # emb_* embeddings
    # y_* class labels
    # g_* protected group labels (one-hot encoded)

print("classes: ", dataset.class_names)
print("protected groups: ", dataset.group_names)

load crowspairs


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
no loss function specified, default to cross entropy
embed test split...
load precomputed embeddings for test set
available splits:  ['test']
classes:  ['stereo', 'antistereo']
protected groups:  ['black', 'white', 'asian', 'poor', 'rich', 'male', 'female', 'physical disability, sickness', 'mental_disability_illness', 'foreigner, immigrant, undocumented', 'canadian', 'us american', 'mexican', 'chinese', 'italian', 'british', 'russian', 'indian', 'homosexual', 'heterosexual', 'overweight', 'underweight', 'fit', 'tall', 'short', 'christian', 'muslim', 'jewish', 'old', 'young']


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model_name = 'meta-llama/Llama-3.2-1B'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=model.config.max_position_embeddings,
                                                       truncation=True)

model = model.to('cuda')

model

LlamaModel(
  (embed_tokens): Embedding(128256, 2048)
  (layers): ModuleList(
    (0-15): 16 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((2048,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding(

In [4]:
import torch
test = ['hello', 'bla', 'yup', 'bla']
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(test, return_tensors='pt', truncation=True, max_length=1024, padding='max_length')

input_ids = inputs['input_ids'].to('cuda')
attention_mask = inputs['attention_mask'].to('cuda')

inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

In [5]:
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Pooling for sentence embeddings

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [23]:
embeddings.shape

torch.Size([4, 2048])

In [2]:
from embedding import BertHuggingface
lm = BertHuggingface(2, model_name='meta-llama/Llama-3.2-1B', batch_size=1, pooling='mean')

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


manually define padding token for model meta-llama/Llama-3.2-1B
Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
no loss function specified, default to cross entropy


In [3]:
lm.embed(['hello how are you today and tomorrow and  bla bal bla', 'bla', 'bla', 'bla', 'bla'])

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
100%|█████████████████████████████████████████████| 5/5 [00:01<00:00,  2.77it/s]


array([[ 0.8398072 ,  4.030428  ,  3.1629562 , ..., -2.4928966 ,
        -3.6599631 , -1.7879707 ],
       [-0.12256873,  1.5742574 ,  4.4297776 , ...,  0.3980987 ,
        -1.3360834 ,  0.11410296],
       [-0.12256873,  1.5742574 ,  4.4297776 , ...,  0.3980987 ,
        -1.3360834 ,  0.11410296],
       [-0.12256873,  1.5742574 ,  4.4297776 , ...,  0.3980987 ,
        -1.3360834 ,  0.11410296],
       [-0.12256873,  1.5742574 ,  4.4297776 , ...,  0.3980987 ,
        -1.3360834 ,  0.11410296]], dtype=float32)